In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Experiments - Manage artifacts lineage.

* [Get started with Vertex AI Experiments](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb?hl=ko#scrollTo=title%3Ageneric%2Cgcp)

### Install Vertex AI SDK for Python and other required packages


In [1]:
! pip3 install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# @title Authentication to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [3]:
# @title Set GCP information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [4]:
# @title Create a bucket.
BUCKET_URI = f"gs://mlops-{PROJECT_ID}-0103"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-ai-hangsik-0103/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-ai-hangsik-0103' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [5]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [6]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://mlops-ai-hangsik-0103/
No changes made to gs://mlops-ai-hangsik-0103/


In [7]:
# @title Import libraries
import os
import uuid

import google.cloud.aiplatform as aiplatform

In [17]:
# @title Initialize Vertex AI SDK for Python
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Artifact lineage in experiment runs

In [18]:
# @title Create artifact lineage in experiment runs

EXPERIMENT_NAME = f"artifact-{uuid.uuid1()}"
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/artifact-528c6804-c972-11ef-ac06-0242ac1c000c-run-1 to Experiment: artifact-528c6804-c972-11ef-ac06-0242ac1c000c


In [19]:
# @title Create a dataset and model artifacts

DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

In [20]:
# @title Create the artifact lineage

with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    aiplatform.log_params({"units": 256})
    aiplatform.log_metrics({"acc": 96.8})

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

In [21]:
# @title Get the experiment results
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

                                                                 0
experiment_name      artifact-528c6804-c972-11ef-ac06-0242ac1c000c
run_name                                                     run-1
run_type                                      system.ExperimentRun
state                                                     COMPLETE
param.units                                                  256.0
metric.acc                                                    96.8
metric.lineage   https://console.cloud.google.com/vertex-ai/loc...


In [22]:
# @title Visualize the artifact lineage

print("Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri)

Open the following link: https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/e42c4a98-c233-4f08-8bbd-6f86fb0585eb?project=ai-hangsik


In [ ]:
# @title Delete the artifact lineage

try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

INFO:google.cloud.aiplatform.base:Deleting Artifact : projects/721521243942/locations/us-central1/metadataStores/default/artifacts/b67dd391-a5da-4273-b298-62df7a3564db
INFO:google.cloud.aiplatform.base:Artifact deleted. . Resource name: projects/721521243942/locations/us-central1/metadataStores/default/artifacts/b67dd391-a5da-4273-b298-62df7a3564db
INFO:google.cloud.aiplatform.base:Deleting Artifact resource: projects/721521243942/locations/us-central1/metadataStores/default/artifacts/b67dd391-a5da-4273-b298-62df7a3564db
INFO:google.cloud.aiplatform.base:Delete Artifact backing LRO: projects/721521243942/locations/us-central1/metadataStores/default/artifacts/b67dd391-a5da-4273-b298-62df7a3564db/operations/3125891113722839040
INFO:google.cloud.aiplatform.base:Artifact resource projects/721521243942/locations/us-central1/metadataStores/default/artifacts/b67dd391-a5da-4273-b298-62df7a3564db deleted.
INFO:google.cloud.aiplatform.base:Deleting Artifact : projects/721521243942/locations/us-c

In [24]:
# @title Delete the experiment
exp = aiplatform.Experiment(EXPERIMENT_NAME)

try:
    exp.delete()
except Exception as e:
    print(e)

To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"artifact-528c6804-c972-11ef-ac06-0242ac1c000c-run-1-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()
INFO:google.cloud.aiplatform.base:Deleting Context : projects/721521243942/locations/us-central1/metadataStores/default/contexts/artifact-528c6804-c972-11ef-ac06-0242ac1c000c-run-1
INFO:google.cloud.aiplatform.base:Context deleted. . Resource name: projects/721521243942/locations/us-central1/metadataStores/default/contexts/artifact-528c6804-c972-11ef-ac06-0242ac1c000c-run-1
INFO:google.cloud.aiplatform.base:Deleting Context resource: projects/721521243942/locations/us-central1/metadataStores/default/contexts/artifact-528c6804-c972-11ef-ac06-0242ac1c000c-run-1
INFO:google.cloud.aiplatform.base:Delete Context backing LRO: 

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
! rm -rf custom

delete_bucket = False

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}